In [1]:
from modelbuilder.model import Model
from modelbuilder.validation.validate import build_metrics

In [2]:
from api.train.train import load_dataset, create_pipeline, get_splitter

In [3]:
dataset = load_dataset()

Index(['ID', 'Price', 'Manufacturer', 'Model', 'Prod__year', 'Category',
       'Leather_interior', 'Fuel_type', 'Engine_volume', 'Mileage',
       'Cylinders', 'Gear_box_type'],
      dtype='object')
         ID  Price   Manufacturer Model  Prod__year   Category  \
0  45798484  51746            GAZ    20        1953      Sedan   
1  45792773   3200            GAZ    20        1953       Jeep   
2  45791976   7840            GAZ    21        1964      Sedan   
3  45812939   5331            GAZ    24        1990  Universal   
4  44828493   4234  MERCEDES-BENZ   190        1989      Sedan   

   Leather_interior Fuel_type  Engine_volume   Mileage  Cylinders  \
0             False    Petrol            2.0       0.0        4.0   
1             False    Petrol            3.2  100000.0        4.0   
2             False    Petrol            2.4       0.0        4.0   
3              True    Petrol            6.3    2000.0        8.0   
4             False    Petrol            2.0       0.0   

In [27]:
pipeline = create_pipeline()
splitter = get_splitter()
# Create object model and fit it
model_name = "car_price_2"
model = Model(model_name, pipeline)

In [28]:
model.fit(dataset)

In [29]:
metrics = build_metrics(model, model.dataset, splitter)

In [7]:
metrics

{'err_prediction_train_mean': 2679.043,
 'abs_err_prediction_train_mean': 13249.709,
 'perc_err_prediction_train_mean': 1706.325,
 'err_prediction_0_train_mean': 5358.085,
 'abs_err_prediction_0_train_mean': 13002.118,
 'perc_err_prediction_0_train_mean': 1587.288,
 'err_prediction_1_train_mean': -0.0,
 'abs_err_prediction_1_train_mean': 13972.736,
 'perc_err_prediction_1_train_mean': 1828.017,
 'err_prediction_mean': 2638.357,
 'abs_err_prediction_mean': 13375.737,
 'perc_err_prediction_mean': 1706.135,
 'err_prediction_0_mean': 5360.49,
 'abs_err_prediction_0_mean': 13017.612,
 'perc_err_prediction_0_mean': 1590.883,
 'err_prediction_1_mean': -83.777,
 'abs_err_prediction_1_mean': 14232.598,
 'perc_err_prediction_1_mean': 1824.135,
 'err_prediction_train_median': -1354.884,
 'abs_err_prediction_train_median': 8393.979,
 'perc_err_prediction_train_median': 51.966,
 'err_prediction_0_train_median': -0.025,
 'abs_err_prediction_0_train_median': 8306.887,
 'perc_err_prediction_0_train_me

In [30]:
from api.helpers.mlflow import register_mlflow
import os
os.environ["MLFLOW_TRACKING_URI"]="http://localhost:5000"
runid = register_mlflow(experiment = model_name, python_model = model, metrics = metrics, params={}, tags = {})

2024/02/27 12:14:35 INFO mlflow.types.utils: Unsupported type hint: <class 'modelbuilder.dataset.Dataset'>, skipping schema inference
2024/02/27 12:14:35 INFO mlflow.types.utils: Unsupported type hint: <class 'modelbuilder.dataset.Dataset'>, skipping schema inference


/home/pablombao/mlops-pricecar/.venv/lib/python3.11/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/home/pablombao/mlops-pricecar/.venv/lib/python3.11/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [13]:
runid

'22cfe3eaf3674ae09172e9f8f2c482bc'

In [15]:
from api.helpers.storage import upload_blob_from_file

upload_blob_from_file("data/car_price_clean.csv", "mlflow-plv", "test","data.csv", delete=False)

In [20]:
import mlflow
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_diabetes
from sklearn.ensemble import RandomForestRegressor
os.environ["MLFLOW_TRACKING_URI"]="http://0.0.0.0:5000"
# set the experiment id
mlflow.set_experiment(experiment_id="132647442831998471")

mlflow.autolog()
db = load_diabetes()

X_train, X_test, y_train, y_test = train_test_split(db.data, db.target)

# Create and train models.
rf = RandomForestRegressor(n_estimators=100, max_depth=6, max_features=3)
rf.fit(X_train, y_train)

# Use the model to make predictions on the test dataset.
predictions = rf.predict(X_test)